# Algorithm for vibration calculation (made by Roman Bokhyan)
In this part I will propose an algorithm that realizes the calculation of vibrations of an object from a video recorded on a special high-speed camera (which allows you to shoot up to several thousand FPS). The algorithm will be implemented in Python using the following libraries.

In [1]:
import numpy as np
from scipy.fft import fft2, ifft2, fftshift
import cv2
import os
from concurrent.futures import ThreadPoolExecutor

## Part 1. Preparing the data
My colleague has prepared special noise-cleared footage of the subject and I will use it for work.

In [ ]:
frames_directory = "preped_data/filtered_event_frames"
video_path = "raw_data/high_speed_cam_videos/MotorAmplif_motion_evm_2022-12-23-10-50-19.mp4"